In [25]:
import pandas as pd
from pathlib import Path

f_stats  = Path("npbstats_batting_with_stint.csv")
f_jp = Path("npb.jp_Batting_with_stint.csv")
f_people = Path("npb_People_combined_cleaned.csv")

stats  = pd.read_csv(f_stats)
jp = pd.read_csv(f_jp)
people = pd.read_csv(f_people)

C:\Users\Acer\AppData\Local\Temp\ipykernel_16952\118141001.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  stats  = pd.read_csv(f_stats)


In [26]:
def clean(series: pd.Series) -> pd.Series:
    return (series.astype("string").str.strip().str.replace(r"\.0$", "", regex=True).replace({"nan": pd.NA}))

for df, col in [
    (stats,"SearchID"), (stats,"Player ID"),
    (jp, "SearchID"), (jp,"Player ID"),
    (people,"SearchID"),(people,"Player ID")]:
    if col in df.columns:
        df[col] = clean(df[col])

In [ ]:
# Player-ID → SearchID
pid2sid = (people[["Player ID","SearchID"]].dropna().drop_duplicates("Player ID", keep="first"))

# SearchID → Player-ID
sid2pid = (people[["SearchID","Player ID"]].dropna().drop_duplicates("SearchID", keep="first"))

jp = jp.merge(pid2sid, on="Player ID", how="left", suffixes=("", "_map"))

if "SearchID_map" in jp.columns:
    jp["SearchID"] = jp["SearchID"].fillna(jp["SearchID_map"])
    jp.drop(columns="SearchID_map", inplace=True) 

stats = stats.merge(sid2pid, on="SearchID", how="left", suffixes=("", "_map"))

if "Player ID_map" in stats.columns:
    stats["Player ID"] = stats["Player ID"].fillna(stats["Player ID_map"])
    stats.drop(columns="Player ID_map", inplace=True)

for df in (stats, jp):
    df["Team"] = (df["Team"].astype("string").str.strip().str.upper())

dup_cols = ["Year", "G", "PA", "AB", "R", "H", "2B", "3B", "HR", "TB", "RBI", "SB"]

In [28]:
def make_key(df):
    return (df[dup_cols].fillna("").astype("string").agg("|".join, axis=1))

stats["__dupkey__"] = make_key(stats)
jp["__dupkey__"] = make_key(jp)
jp = jp[~jp["__dupkey__"].isin(set(stats["__dupkey__"]))]


merged = (pd.concat([stats, jp], ignore_index=True).drop(columns="__dupkey__"))
need_sid = merged["SearchID"].isna()
if need_sid.any():
    merged.loc[need_sid, "SearchID"] = (merged.loc[need_sid, "Player ID"].map(pid2sid.set_index("Player ID")["SearchID"]))


In [29]:
still_missing = merged["SearchID"].isna().sum()
if still_missing:
    print(f"Check {still_missing}")

merged["__sid_num"] = pd.to_numeric(merged["SearchID"], errors="coerce")
merged = (merged.sort_values(["__sid_num","Year","Team","Stint"]).drop(columns="__sid_num").reset_index(drop=True))
out_file = "npb_batting_merged10_test.csv"
merged.to_csv(out_file, index=False)

Check 422


In [ ]:
import pandas as pd
from pathlib import Path

INPUT_FILE = Path("npb_batting_merged10.csv")
OUTPUT_FILE = INPUT_FILE.with_name(INPUT_FILE.stem + "all_filled.csv")

df = pd.read_csv(INPUT_FILE, low_memory=False)

team2league = (df[df["League"].isin(["CL", "PL"])].groupby(["Team", "League"]).size().unstack(fill_value=0).idxmax(axis=1).to_dict())

blank_mask = df["League"].isna() | (df["League"].astype(str).str.strip() == "")
df.loc[blank_mask, "League"] = df.loc[blank_mask, "Team"].map(team2league)

still_blank = df["League"].isna() | (df["League"].astype(str).str.strip() == "")
print(still_blank)

df.to_csv(OUTPUT_FILE, index=False)